In [ ]:
import pandas as pd
import numpy as np
import sys

In [20]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [21]:
def align_and_append_sheet(template, other_sheet):
    alignment = template.align(other_sheet, join="outer")[1]
    return template.append(alignment).dropna(axis=0, how='all')

In [22]:
def generate_patient_id(model_name):
    patient_id = f'{model_name}_patient'
    suffix = model_name[-2:]
    if suffix == "-B" or suffix == "-A":
        patient_id = f'{model_name[:-2]}_patient'
    return patient_id
    

In [40]:
def generate_sample_id(model_name):
    return f'{model_name}_sample'

In [41]:
def generate_patient_sheet(hcmiDf, patient_template):
    patient_columns = ["Gender","Race", "Age At Diagnosis (Years)", "Name"]
    column_name_map = {"Gender": "sex", "Race" : "ethnicity","Age At Diagnosis (Years)" :"age_at_initial_diagnosis", "Name": "patient_id"}
    HCMI_patient_columns = hcmiDf.loc[:, patient_columns].rename(columns=column_name_map)
    HCMI_patient_columns["patient_id"] = HCMI_patient_columns["patient_id"].apply(generate_patient_id)
    align_and_append_sheet(patient_template, HCMI_patient_columns).to_csv("HCMI_metadata-patient.tsv", sep='\t', index=False)

In [55]:
def generate_sample_sheet(hcmiDf, samplet_template):
    sample_columns = ["Name", "Primary Site", "Clinical Tumor Diagnosis", "Histological Subtype", "Tissue Status", "Acquisition Site", "Age At Acquisition (Years)", "TNM Stage", "Histological Grade"]
    sample_name_map = { "Name": "model_id", "Primary Site" : "primary_site", "Tissue Status" : "tumour_type", "Acquisition Site" : "collection_site", "Age At Acquisition (Years)" : "age_in_years_at_collection", "TNM Stage" : "stage", "Histological Grade" : "grade" }
    HCMI_sample_columns = hcmiDf.loc[:, sample_columns].rename(columns=sample_name_map)
        HCMI_sample_columns["patient_id"] = HCMI_sample_columns['model_id'].apply(generate_patient_id)
    HCMI_sample_columns["sample_id"] = HCMI_sample_columns['model_id'].apply(generate_sample_id)
    HCMI_sample_columns["diagnosis"] = HCMI_sample_columns["Clinical Tumor Diagnosis"] + " " + HCMI_sample_columns["Histological Subtype"]
    HCMI_sample_columns = HCMI_sample_columns.drop(["Clinical Tumor Diagnosis","Histological Subtype"], axis=1)
    HCMI_sample_columns["staging_system"] = "TNM staging system"
    HCMI_sample_columns['sharable'] = 'yes'
    accepted_tumour_types = ["Metastasis", "Primary", "Recurrent"]
    HCMI_sample_columns = HCMI_sample_columns[HCMI_sample_columns.loc[:,'tumour_type'].isin(accepted_tumour_types)]
    align_and_append_sheet(sample_template, HCMI_sample_columns).to_csv("HCMI_metadata-sample.tsv", sep='\t', index=False)

In [23]:
hcmiDf = pd.read_csv("model-table.tsv", sep='\t')
patient_template = pd.read_csv("metadata_template-patient.tsv", sep='\t').dropna(axis=0, how='all')
sample_template = pd.read_csv("metadata_template-sample.tsv", sep='\t').dropna(axis=0, how='all')

In [67]:
used_columns_to_drop = ["Gender","Race", "Age At Diagnosis (Years)","Primary Site", "Clinical Tumor Diagnosis", "Histological Subtype", "Tissue Status", "Acquisition Site", "Age At Acquisition (Years)", "TNM Stage", "Histological Grade"]
hcmiDf.drop(used_columns_to_drop, axis=1).to_excel("leftovers.xlsx", index=False)